**reference** <br>
 : https://github.com/niconielsen32/ComputerVision/tree/master/StereoVisionDepthEstimation 

import module

In [2]:
import numpy as np
import cv2 
import glob

### checkboard calib image capture

In [ ]:
cap1 = cv2.VideoCapture(1) # 맞는 웹캠 번호 입력
cap2 = cv2.VideoCapture(2)

num = 0

while cap1.isOpened():

    succes1, img1 = cap1.read()
    succes2, img2 = cap2.read()

    k = cv2.waitKey(5)

    if k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('../images/calibrationStereo/stereoLeft/calibL' + str(num) + '.png', img1) # 이미지 저장 경로 설정 
        cv2.imwrite('../images/calibrationStereo/stereoRight/calibR' + str(num) + '.png', img2)
        print("images saved!")
        num += 1

    cv2.imshow('CAM 1',img1)
    cv2.imshow('CAM 2',img2)
    
    # Hit "q" to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release and destroy all windows before termination
cap1.release()
cap2.release()

cv2.destroyAllWindows()

images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!
images saved!


**(+) image quality 개선** <br>

In [23]:
# 적용할 개선 과정을 함수로 정의합니다.
def improve_image(image_path):
    # 이미지를 불러옵니다.
    img = cv2.imread(image_path, 0)  # 0 flag for grayscale

    # Histogram Equalization 적용
    img_he = cv2.equalizeHist(img)

    # Adaptive Histogram Equalization 적용
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_ache = clahe.apply(img)

    # Unsharp Masking 적용
#     gaussian = cv2.GaussianBlur(img_ache, (9,9), 10.0) # You can change the kernel size and standard deviation
    gaussian = cv2.GaussianBlur(img_ache, (5,5), 5.0)
    """
        커널 사이즈를 크게 하거나 표준편차를 높이면 블러 처리의 정도가 강해지므로, 선명도가 더욱 높아집니다. 
        그러나 너무 높게 설정하면 이미지의 디테일이 손실될 수 있으므로 주의가 필요합니다.
    """
#     img_unsharp = cv2.addWeighted(img_ache, 1.5, gaussian, -0.5, 0, img_ache) 
    img_unsharp = cv2.addWeighted(img_ache, 2.0, gaussian, -0.5, 0, img_ache) 
    """
        여기서 1.5와 -0.5는 각각 원본 이미지와 블러 이미지에 부여하는 가중치를 의미합니다. 
        이 두 값의 차이가 클수록 선명도가 높아집니다. 
        즉, 원본 이미지에 더 높은 가중치를 부여하거나 블러 이미지에 더 낮은 가중치를 부여하면 선명도가 더욱 높아집니다.
    """

    # 밝기 증가
    img_bright = cv2.convertScaleAbs(img_unsharp, alpha=1, beta=-20) # You can change alpha for contrast control, beta for brightness

    return img_bright

In [24]:
# 0부터 11까지 반복합니다.
for num in range(12):
    image_path = '../images/calibrationStereo/stereoLeft/calibL' + str(num) + '.png'
    # 이미지 품질 개선 과정을 진행합니다.
    improved_image = improve_image(image_path)
    # 개선된 이미지를 저장합니다.
    cv2.imwrite('../images/calibrationStereo/stereoLeft/improved_calibL' + str(num) + '.png', improved_image)
    
    image_path = '../images/calibrationStereo/stereoRight/calibR' + str(num) + '.png'
    # 이미지 품질 개선 과정을 진행합니다.
    improved_image = improve_image(image_path) 
    # 개선된 이미지를 저장합니다.
    cv2.imwrite('../images/calibrationStereo/stereoRight/improved_calibR' + str(num) + '.png', improved_image)

### stereo calibration 

: save calibration parameters to **stereoMap.xml** 

In [4]:
################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize =  (7,6) # (8,6) 
frameSize = (640, 480)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

objp = objp * 22 ## ---------------------

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.

# imagesLeft = glob.glob('images/calibrationStereo/stereoLeft/*.png')
# imagesRight = glob.glob('images/calibrationStereo/stereoRight/*.png')
imagesLeft = glob.glob('../images/calibrationStereo/stereoLeft/improved_*.png')
imagesRight = glob.glob('../images/calibrationStereo/stereoRight/improved_*.png')

for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    print(".")

    imgL = cv2.imread(imgLeft)
    imgR = cv2.imread(imgRight)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('grayL', grayL)
    cv2.imshow('grayR', grayR)

    # Find the chess board corners
    retL, cornersL = cv2.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, chessboardSize, None)

    # ================================================================ test code 
    # After trying to find the corners
    retL, cornersL = cv2.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, chessboardSize, None)

    # Check if corners were found and print the result
    if retL and retR:
        print(f"Corners were found in the image: {imgLeft}")
    else:
        print(f"Corners were not found in the image: {imgLeft}")
    # =====================================================================
    
    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv2.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv2.imshow('img left', imgL)
        cv2.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv2.imshow('img right', imgR)
        cv2.waitKey(1000)

cv2.destroyAllWindows()


############## CALIBRATION #######################################################

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv2.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv2.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))


########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)

#print(newCameraMatrixL)
#print(newCameraMatrixR)

########## Stereo Rectification #################################################

rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv2.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv2.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv2.CV_16SC2)
stereoMapR = cv2.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv2.CV_16SC2)

print("Saving parameters!")
cv_file = cv2.FileStorage('stereoMap.xml', cv2.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()

.
Corners were not found in the image: images/calibrationStereo/stereoLeft\improved_calibL0.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL1.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL10.png
.
Corners were not found in the image: images/calibrationStereo/stereoLeft\improved_calibL11.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL2.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL3.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL4.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL5.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL6.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft\improved_calibL7.png
.
Corners were found in the image: images/calibrationStereo/stereoLeft

### calibration result TEST 

: **undistortRectify** by **stereoMap.xml** information 

In [39]:
import calibration

In [41]:
# Open both cameras
cap_right = cv2.VideoCapture(2, cv2.CAP_DSHOW)                    
cap_left =  cv2.VideoCapture(1, cv2.CAP_DSHOW)

# Main loop for depth estimation using stereo vision
while(cap_right.isOpened() and cap_left.isOpened()):    
    succes_right, frame_right = cap_right.read()
    succes_left, frame_left = cap_left.read()

    # Show the frames
    cv2.imshow("frame right", frame_right) 
    cv2.imshow("frame left", frame_left)
    
    ################## CALIBRATION #########################################################

    frame_right, frame_left = calibration.undistortRectify(frame_right, frame_left) 

    ########################################################################################

    # Show the frames
    cv2.imshow("frame right - undistortRectified", frame_right) 
    cv2.imshow("frame left - undistortRectified", frame_left)
    
    # Hit "q" to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release and destroy all windows before termination
cap_right.release()
cap_left.release()

cv2.destroyAllWindows()

In [3]:
cv2.__version__

'4.8.0'